## Графовые нейронные сети (GNNs)

Графовые нейронные сети представляют собой класс нейронных сетей, специально разработанный для работы с графовыми структурами данных. Они могут моделировать зависимости и взаимодействия между сущностями в графе, позволяя учитывать контекст и структуру данных при выполнении задач.

В GNN каждая вершина графа обновляется на основе её соседей, что позволяет модели учить представления вершин, учитывая их окружение. Такие модели широко применяются в задачах, связанных с анализом социальных сетей, биологических сетей, рекомендательных систем и графовых знаний.

## Graph embedding
Эмбеддинги в контексте графовых нейронных сетей представляют собой векторные представления вершин графа. Они извлекаются из GNN и представляют сущности в пространстве низкой размерности. Эти векторы обладают свойством сохранять структурную информацию о графе и могут использоваться для различных задач, таких как предсказание отношений или классификация вершин.

## Тестирование и Валидация

Тестирование и валидация важны для оценки работы графовых нейронных сетей.

Тестирование позволяет оценить обобщающую способность модели на новых данных, которые не участвовали в обучении. Это важно для проверки, насколько хорошо модель обобщает свои знания на новые сценарии.

Валидация используется для подбора оптимальных гиперпараметров модели. Настройка параметров происходит на основе производительности модели на отложенной валидационной выборке.

### Установка зависимостей

Для работы с графовыми нейронными сетями будем использовать [Pykeen](https://pykeen.readthedocs.io/en/stable/). Необходимо импортировать соответствующую библиотеку. В данном шаге мы устанавливаем Pykeen. Этот шаг включает установку библиотеки Pykeen с использованием команды !pip install pykeen. Pykeen предоставляет реализацию различных моделей графовых знаний и инструменты для работы с ними.

In [ ]:
# Установка Pykeen
!pip install pykeen

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.3/739.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 11.2 MB/s eta 0:00:00
  Created wheel for rexmex: filename=rexmex-0.1.3-py3-none-any.whl size=19952 sha256=43cef70dfb1b6f836e54f0ce3ca40565c89ad631e2f47bf4f79407f35c7d8fa1
  Stored in directory: /root/.cache/pip/wheels/13/31/9d/51fb64454e6012bc88dd72b57609646432e478ba35772736d6
Successfully built rexmex


Далее, мы импортируем библиотеку Pykeen для использования её функциональности в нашем коде.

In [ ]:
import pykeen

После выполнения этого шага, мы готовы к использованию Pykeen для создания, обучения и оценки моделей графовых знаний.

## Взаимодействие с графовым датасетом и описание данных

В данной работе мы будем использовать графовый датасет [PharmKG](https://pykeen.readthedocs.io/en/stable/api/pykeen.datasets.PharmKG.html).


Импорт необходимого модуля Pykeen. Мы импортируем модуль datasets из Pykeen для работы с графовыми датасетами.

In [ ]:
from pykeen.datasets import PharmKG

INFO:pykeen.utils:Using opt_einsum


Загружаем графовый датасет Nations из Pykeen. В результате выполнения этой команды будет создан объект, представляющий графовый датасет.

In [ ]:
# Загрузка графового датасета PharmKG
pharm_dataset = PharmKG()

Мы можем вывести описание датасета, чтобы получить информацию о количестве сущностей, отношений и других характеристиках.

In [ ]:
# Просмотр описания датасета
print(pharm_dataset)

INFO:pykeen.datasets.base:downloading data from https://zenodo.org/record/4077338/files/raw_PharmKG-180k.zip to /root/.data/pykeen/datasets/pharmkg/raw_PharmKG-180k.zip


INFO:pykeen.datasets.base:reordering columns: ['Entity1_name', 'relationship_type', 'Entity2_name']
INFO:pykeen.triples.splitting:done splitting triples to groups of sizes [691412, 109324, 109324]


PharmKG(num_entities=188296, num_relations=39, create_inverse_triples=False)


#### Получение тренировочных данных
Мы можем получить тренировочные факты, представленные в виде троек сущность-отношение-сущность.
Таким образом, после выполнения этих шагов, у нас есть загруженный графовый датасет, и мы можем использовать его для обучения и тестирования моделей графовых знаний в Pykeen.

In [ ]:
# Получение тренировочных фактов (троек сущность-отношение-сущность)
training_data = pharm_dataset.training.mapped_triples
print(training_data[:5])

tensor([[     0,      2, 161966],
        [     0,     21,  26679],
        [     1,     17,  95159],
        [     2,     31,  23633],
        [     3,      3,  65653]])


## Описание базовой задачи

В данном контексте базовой задачей является предсказание отношений между сущностями в графовом датасете. Каждая тройка в графе представляет отношение между двумя сущностями. Например, в социальной сети, где сущности - пользователи и страницы, отношения могут быть "пользователь подписан на страницу".

In [ ]:
# Импорт необходимых модулей Pykeen
from pykeen.pipeline import pipeline


#### Выбор модели
Мы выбираем модель для решения задачи предсказания отношений. В данном случае, мы используем модель [TransE](https://paperswithcode.com/method/transe), одну из популярных моделей для этой задачи.

#### Выбор датасета
Мы указываем графовый датасет, с которым будем работать. В данном примере - nations_dataset, который содержит данные о связях в между странами.

#### Количество эпох обучения
Задаем количество эпох обучения (итераций по всему датасету).

Проверяем наличие GPU устройства и вызываем функцию pipeline(), которая запускает процесс обучения модели, тестирования и валидации.

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Задаем параметры для pipeline
pipeline_results = pipeline(
    model = "TransE",      # Выбираем модель (TransE - одна из моделей для предсказания отношений)
    dataset = pharm_dataset,     # Выбираем графовый датасет (Nations)
    training_kwargs = dict(num_epochs=20),   # Количество эпох обучения
    device = device
)

INFO:pykeen.pipeline.api:Using device: cuda


Training epochs on cuda:0:   0%|          | 0/20 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

...

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

Training batches on cuda:0:   0%|          | 0/3417 [00:00<?, ?batch/s]

INFO:pykeen.evaluation.evaluator:Starting batch_size search for evaluation now...
INFO:pykeen.evaluation.evaluator:Concluded batch_size search with batch_size=128.


Evaluating on cuda:0:   0%|          | 0.00/109k [00:00<?, ?triple/s]

INFO:pykeen.evaluation.evaluator:Evaluation took 210.17s seconds


In [ ]:
# Выводим результаты
print(pipeline_results)

PipelineResult(random_seed=995784644, model=TransE(
  (loss): MarginRankingLoss(
    (margin_activation): ReLU()
  )
  (interaction): TransEInteraction()
  (entity_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(188296, 50)
    )
  )
  (relation_representations): ModuleList(
    (0): Embedding(
      (_embeddings): Embedding(39, 50)
    )
  )
  (weight_regularizers): ModuleList()
), training=TriplesFactory(num_entities=188296, num_relations=39, create_inverse_triples=False, num_triples=874588, path="/root/.data/pykeen/datasets/pharmkg/raw_PharmKG-180k.zip"), training_loop=<pykeen.training.slcwa.SLCWATrainingLoop object at 0x782fe13db760>, losses=[0.681531792038325, 0.261174362236762, 0.16822145487878767, 0.1297931945082787, 0.10908684827646806, 0.09515525557848666, 0.08455729397285794, 0.07684351447920615, 0.07048696856253461, 0.06594348690677777, 0.06263050784481995, 0.058561527985454755, 0.055732770815583255, 0.05301449329254046, 0.051458372467109985,

### Triple Scoring

In [ ]:
from pykeen.predict import predict_triples

Рассчитаем оценки для всех validation троек из набора данных, на котором мы обучали модель.

In [ ]:
pack = predict_triples(model=pipeline_results.model, triples=pharm_dataset.validation)
df = pack.process(factory=pipeline_results.training).df



```
# This is formatted as code
```

Показаны тройки, получившие наивысший score

In [ ]:
df.nlargest(n=5, columns="score")

,head_id,head_label,relation_id,relation_label,tail_id,tail_label,score
1719,171786,tec,27,Rg,171786,tec,-0.683091
2245,84549,hcn1,27,Rg,84549,hcn1,-0.683091
39278,117401,mylk3,27,Rg,117401,mylk3,-0.683091
57551,124540,nme1,27,Rg,124540,nme1,-0.683091
58367,17108,abcd3,27,Rg,17108,abcd3,-0.683091


В рамках самостоятельной работы попробуйте обучить и протестировать модель на любом другом датасете доступном на https://pykeen.readthedocs.io/en/stable/reference/datasets.html.